In [1]:
import nibabel as nb
import numpy as np
from skimage.measure import label
import glob
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def getLargestCC(segmentation):
    labels = label(segmentation)
    assert( labels.max() != 0 ) # assume at least 1 CC
    largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
    return largestCC

def create_if_not(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
df = pd.read_csv('/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/0_0.0_concat_report_final.csv')

with open('/home/abhijit/Jyotirmay/my_thesis/dataset_groups/whole_body_datasets/UKB/test_volumes.txt', mode='r') as file_handler:
    volumes = file_handler.read().splitlines()

projects = ['MC_dropout_quicknat', 'probabilistic_quicknat']  #'hierarchical_quicknat', 'full_bayesian'

dicts = {}
for p in projects:
    dicts[p] = {}
    for ijk, v in enumerate(volumes):
        print("\rProcessing {} {}/{}.".format(p, ijk, len(volumes)), end="")
        try: 
            if p == 'MC_dropout_quicknat':
                pfix = df[df['volume_id']==v]['target_scan_file'].values[0].split('.')[0][-1]
            else:
                pfix = ''

            data_nifty = nb.load(f'/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/{p}_UKB_v2/{p}_UKB_v2_predictions_UKB/{v}{pfix}_seg.nii.gz')

            header = data_nifty.header
            data = data_nifty.get_fdata()

            liver_seg =np.zeros_like(data)
            spleen_seg = np.zeros_like(data)

            cc = getLargestCC(data)
            liver_seg[cc] = 2

            data[cc] = 0
            if np.unique(data)[1] != 1:
                print('No spleen here!!! ')
                continue

            ss = getLargestCC(data)
            spleen_seg[ss] = 1

            gt = (liver_seg + spleen_seg).astype('float32')

            nifti_img = nb.MGHImage(np.squeeze(gt), np.eye(4))
            path = f'/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/{p}_UKB_v2/{p}_UKB_v2_predictions_UKB_post_processed'
            create_if_not(path)
            nb.save(nifti_img, os.path.join(path, f'{v}_seg.nii.gz'))
        except Exception as e:
            print(e)
            continue

Processing MC_dropout_quicknat 18/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 25/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 31/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 51/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 68/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 79/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 118/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 175/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 188/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 210/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 233/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quick